### Import Knet & ONNX

In [1]:
using Knet;

In [3]:
include(Knet.dir("data","mnist.jl"))
dtrn,dtst = mnistdata();

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Pkg.jl:531


LoadError: expected package `CodecZlib [944b1d66]` to exist in the manifest (use `resolve` to populate the manifest)

In [ ]:
KnetOnnx_PATH = "/Users/syny/kafamagore/KnetOnnx.jl/src"
push!(LOAD_PATH, KnetOnnx_PATH)
using KnetOnnx;

### Read ONNX Graph

In [2]:
# file_path = "cnn.onnx"
file_path = "/Users/syny/Downloads/bvlcalexnet-9.onnx"
graph = ONNXtoGraph(file_path);

In [7]:
graph

KnetOnnx.Types.Graph(Any[KnetOnnx.Types.Node(AbstractString["input.1", "conv1.weight", "conv1.bias"], AbstractString["9"], "Conv_0", "Conv", "", Dict{Any,Any}(:dilations => [1, 1],:group => 1,:pads => [0, 0, 0, 0],:kernel_shape => [3, 3],:strides => [1, 1]), ""), KnetOnnx.Types.Node(AbstractString["9"], AbstractString["10"], "Relu_1", "Relu", "", Dict{Any,Any}(), ""), KnetOnnx.Types.Node(AbstractString["10", "conv2.weight", "conv2.bias"], AbstractString["11"], "Conv_2", "Conv", "", Dict{Any,Any}(:dilations => [1, 1],:group => 1,:pads => [0, 0, 0, 0],:kernel_shape => [3, 3],:strides => [1, 1]), ""), KnetOnnx.Types.Node(AbstractString["11"], AbstractString["12"], "MaxPool_3", "MaxPool", "", Dict{Any,Any}(:pads => [0, 0, 0, 0],:kernel_shape => [2, 2],:strides => [2, 2]), ""), KnetOnnx.Types.Node(AbstractString["12"], AbstractString["13"], "Flatten_4", "Flatten", "", Dict{Any,Any}(:axis => 1), ""), KnetOnnx.Types.Node(AbstractString["13", "fc1.weight", "fc1.bias"], AbstractString["14"], "G

In [3]:
PrintGraph(graph)

model inputs: ["data_0", "conv1_w_0", "conv1_b_0", "conv2_w_0", "conv2_b_0", "conv3_w_0", "conv3_b_0", "conv4_w_0", "conv4_b_0", "conv5_w_0", "conv5_b_0", "fc6_w_0", "fc6_b_0", "fc7_w_0", "fc7_b_0", "fc8_w_0", "fc8_b_0", "OC2_DUMMY_1"]
model outputs: ["prob_1"]
(op1) Conv
	input1: data_0
	input2: conv1_w_0
	input3: conv1_b_0
	output1: conv1_1
(op2) Relu
	input1: conv1_1
	output1: conv1_2
(op3) LRN
	input1: conv1_2
	output1: norm1_1
(op4) MaxPool
	input1: norm1_1
	output1: pool1_1
(op5) Conv
	input1: pool1_1
	input2: conv2_w_0
	input3: conv2_b_0
	output1: conv2_1
(op6) Relu
	input1: conv2_1
	output1: conv2_2
(op7) LRN
	input1: conv2_2
	output1: norm2_1
(op8) MaxPool
	input1: norm2_1
	output1: pool2_1
(op9) Conv
	input1: pool2_1
	input2: conv3_w_0
	input3: conv3_b_0
	output1: conv3_1
(op10) Relu
	input1: conv3_1
	output1: conv3_2
(op11) Conv
	input1: conv3_2
	input2: conv4_w_0
	input3: conv4_b_0
	output1: conv4_1
(op12) Relu
	input1: conv4_1
	output1: conv4_2
(op13) Conv
	input1: conv4_2

In [4]:
model = KnetModel(file_path);

ONNX Operation not yet implemented: LRN


MethodError: MethodError: no method matching iterate(::Nothing)
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:603
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:603
  iterate(!Matched::ExponentialBackOff) at error.jl:253
  ...

In [10]:
function get_ModelLayers(g)
    ModelLayers = []
    for node in g.node; push!(ModelLayers, ModelLayer(node, g)); end
    return ModelLayers
end

get_ModelLayers(graph)

UndefVarError: UndefVarError: ModelLayer not defined

In [5]:
KnetModel.TensorDict(graph)

ErrorException: type DataType has no field TensorDict

### Re-construct model in Knet

In [6]:
model.tensors

Dict{Any,Any} with 11 entries:
  "12"      => Nothing
  "input.1" => Nothing
  "11"      => Nothing
  "13"      => Nothing
  "15"      => Nothing
  "16"      => Nothing
  "14"      => Nothing
  "17"      => Nothing
  "10"      => Nothing
  "18"      => Nothing
  "9"       => Nothing

In [7]:
x,y = first(dtrn)
model(x)

UndefVarError: UndefVarError: dtrn not defined

In [9]:
model.tensors

Dict{Any,Any} with 9 entries:
  "16"      => [0.122901 0.129133 … 0.129186 0.11251; 0.0393511 0.0247273 … 0.0…
  "12"      => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.116799; … ; 0.0 0.0 ……
  "10"      => Float32[0.238287 0.238287 … 0.238287 0.238287; 0.238287 0.238287…
  "15"      => [0.0822569 0.0 … 0.0906895 0.0999589; 0.0 0.0 … 0.0 0.0; … ; 0.0…
  "input.1" => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …
  "9"       => Float32[0.238287 0.238287 … 0.238287 0.238287; 0.238287 0.238287…
  "11"      => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …
  "13"      => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …
  "14"      => [0.0822569 -0.00749586 … 0.0906895 0.0999589; -0.0728656 -0.0673…

In [10]:
@show accuracy(model, dtst)

accuracy(model, dtst) = 0.1136


0.1136

### Re-train the model in Knet

In [11]:
EPOCH = 2
progress!(sgd(model, repeat(dtrn,EPOCH)))
@show accuracy(model, dtst)

┌ Warning: repeat(d::Data,n) is deprecated, use IterTools.ncycle instead.
└ @ Knet /Users/syny/.julia/packages/Knet/bTNMd/src/data.jl:92



Stacktrace:
 [1] getindex(::Array{String,1}, ::Int64) at ./array.jl:788
 [2] (::KnetModel)(::Array{Float32,4}, ::Vararg{Any,N} where N) at /Users/syny/kafamagore/KnetOnnx.jl/src/KnetModel.jl:121
 [3] (::Knet.var"#693#694"{Knet.Minimize{Knet.Repeat},Tuple{Array{Float32,4},Array{UInt8,1}}})() at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:205
 [4] differentiate(::Function; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:144
 [5] differentiate at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:135 [inlined]
 [6] iterate at /Users/syny/.julia/packages/Knet/bTNMd/src/train.jl:23 [inlined]
 [7] iterate at /Users/syny/.julia/packages/Knet/bTNMd/src/progress.jl:69 [inlined]
 [8] progress!(::Knet.Minimize{Knet.Repeat}; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /Users/syny/.julia/packages/Knet/bTNMd/src/progress.jl:58
 [9] progress!(::Knet.Minimize{Knet.Repeat})

BoundsError: BoundsError: attempt to access 1-element Array{String,1} at index [2]

In [ ]:
using Images, Plots
function predictMNIST(model, data)
    i = rand(1:100)
    x = reshape(rand(dtrn)[1][:,:,:,i], 28, 28, 1, 1)
    pred = argmax(model(x), dims=1)[1][1]; if pred==10; pred=0; end
    image = x[:,:,1,1]    
    plot(heatmap(permutedims(image), yflip = true, title = ("Model Output: " * string(pred))))
end
predictMNIST(model, dtst)